In [1]:
!pip install sympy

In [2]:
import os
import sys
from sympy import *
init_printing()

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
def get_degree(el):#только для массивов без элементов в виде чисел
    '''
    Эта функция используется для сортировки членов интеграла по степеням,
    а то там порядок сбивается, и отрицательные члены сначала идут
    '''
    if ask(Q.extended_real(el)):
        return el
    arg = el.args
    if len(arg) > 1:
        return get_degree(arg[-1])
    if len(arg) == 0:
        return 1

In [5]:
cosh.__name__='ch'
sinh.__name__='sh'

In [6]:
x = Symbol('x')
y = Symbol('y')

# КАК ПИСАТЬ ВЫРАЖЕНИЯ

е в степени

In [7]:
exp(-2 * x ** 2)

     2
 -2⋅x 
ℯ     

Дробь, где только циферки

In [8]:
Rational(1, 2)

1/2

Так эти дроби писать не надо!

In [9]:
1 / 2

0.5

Дробь с букавками

In [10]:
exp(-2 * x ** 2) / sqrt(x)

     2
 -2⋅x 
ℯ     
──────
  √x  

Корень произвольной степени. Знаменатель и есть степень корня. Вспоминайте математику

In [11]:
(x + 1) ** Rational(1, 3)

3 _______
╲╱ x + 1 

# Вставьте здесь ваши параметры

In [31]:
x0 = 0 #в этой задаче он должен быть 0, поэтому не меняем
accuracy = 0.001 # до производной какой степени считать
n = 3 #количество. Минимум 2 должно быть
a = 0.25# a в интеграле(верхняя циферка). Нижняя у всех 0
expr = (x) ** Rational(1, 2)*sin(x**2)#<------- здесь должно быть ваше выражение
print('Проверьте, что ваше выражение корректно отображается!')
Integral(expr, (x, 0, a))
print('Вычислить с точностью =', accuracy)

Проверьте, что ваше выражение корректно отображается!


0.25              
 ⌠                
 ⎮         ⎛ 2⎞   
 ⎮   √x⋅sin⎝x ⎠ dx
 ⌡                
 0                

Вычислить с точностью = 0.001


In [35]:
good_detominator = True
for el in expr.args:
    degree = get_degree(el)
    if (degree is None or ask(Q.extended_real(el))) or degree > 1:
        continue
    denominator = diff(el, x)
    if not ask(Q.extended_real(denominator.subs(x, 0))):
        print('В знаменателе только x, поэтому его мы игнорируем при дифференцировании, а потом, получив дифференциал, поделием его на этот знаменатель')
        print('В коде ниже будет результат вместе с делением')
        new_expr = expr / denominator
        new_expr
        good_detominator = False
        break

В знаменателе только x, поэтому его мы игнорируем при дифференцировании, а потом, получив дифференциал, поделием его на этот знаменатель
В коде ниже будет результат вместе с делением


       ⎛ 2⎞
2⋅x⋅sin⎝x ⎠

In [36]:
denominator

 1  
────
2⋅√x

# Вычисление по точности

но Виклянчук сказала, что нужно минимум 4 члена высчитать, поэтому минимум 4 члена и выведется

In [29]:
print('Выражение')
expr
i = 0
while True:
    print(i, 'порядок')
    if good_detominator:
        expr_dif = expr.diff(x, i)#дифференциал
    else:
        expr_dif = new_expr.diff(x, i)
    print('Дифференциал')
    Eq(y, simplify(expand(expr_dif)))#выводим его
    #Eq(y, expr_dif)
    print('При x =', x0)
    Eq(y, expr_dif.subs(x, x0))#выводим чему равен при подстановке
    ser = expr.series(x0=x0, n=i + 1).removeO()#ряд
    integral = sorted(integrate(ser, (x)).args, key=get_degree)# интеграл от ряда
    if len(integral) >= 2:
        l1 = integral[-1].subs(x, a)# последний член интеграла
        l2 = integral[-2].subs(x, a)# предпоследний
        cur_acc = abs(l2 + l1)#считаем здесь их точность
        if cur_acc < accuracy and len(integral) >= 4:
            if not good_detominator:#костыль из-за того, что знаменатель пропадает, и на 1 порядок меньше выводится
                print(i + 1, 'порядок')
                expr_dif = new_expr.diff(x, i + 1)
                print('Дифференциал')
                Eq(y, expr_dif)#выводим его
                print('При x =', x0)
                Eq(y, expr_dif.subs(x, x0))#выводим чему равен при подстановке
            print('\nФинал\n')
            print('Дифференциал')
            ser
            print('Интегралыч')
            integrate(ser, (x))
            integ_need = Add(*integral[:-2])# нужная часть интеграла для итогового счёта
            answer = integ_need.subs(x, a)
            print('Убираем последние 2 члена т.к. они дают меньше нужной точности')
            print('При a =', a)
            Eq(Add(integ_need), answer) 
            try:
                print('Численный ответ = ', answer.p / answer.q)
            except AttributeError:
                print('Численный ответ = ', answer)
            try:
                print('Точность',  cur_acc.p / cur_acc.q)
            except AttributeError:
                print('Точность', cur_acc)
            break
    i +=1

Выражение


      ⎛ 2⎞
√x⋅sin⎝x ⎠

0 порядок
Дифференциал


          ⎛ 2⎞
y = √x⋅sin⎝x ⎠

При x = 0


...

Интегралыч


   31/2    23/2    15/2      7/2
  x       x       x       2⋅x   
- ───── + ───── - ───── + ──────
  78120    1380     45      7   

Убираем последние 2 члена т.к. они дают меньше нужной точности
При a = 0.25


   15/2      7/2                      
  x       2⋅x                         
- ───── + ────── = 0.00223146468874008
    45      7                         

Численный ответ =  0.00223146468874008
Точность 8.63775823067684e-11


# Вычисление по количеству членов

In [20]:
print('Выражение')
expr
i = 0
while True:
    print(i, 'порядок\n')
    if good_detominator:
        expr_dif = expr.diff(x, i)#дифференциал
    else:
        expr_dif = new_expr.diff(x, i)
    print('Дифференциал')
    Eq(y, expr_dif)
    print('При x =', x0)
    Eq(y, expr_dif.subs(x, x0))
    ser = expr.series(x0=x0, n=i + 1).removeO()#ряд
    integral = sorted(integrate(ser, (x)).args, key=get_degree)# интеграл от ряда
    if len(integral) == n:
        l1 = integral[-1].subs(x, a)# последний член интеграла
        l2 = integral[-2].subs(x, a)# предпоследний
        cur_acc = abs(l2 + l1)# берём модуль
        if not good_detominator:#костыль из-за того, что знаменатель пропадает, и на 1 порядок меньше выводится
            print(i + 1, 'порядок')
            expr_dif = new_expr.diff(x, i + 1)
            print('Дифференциал')
            Eq(y, expr_dif)#выводим его
            print('При x =', x0)
            Eq(y, expr_dif.subs(x, x0))#выводим чему равен при подстановке
        print('\nФинал\n')
        print('Дифференциал')
        ser
        print('Интегралыч')
        integ_need = Add(*integral)# нужная часть интеграла для итогового счёта
        integ_need
        answer = integ_need.subs(x, a)
        print('При a =', a)
        Eq(Add(integ_need), answer) 
        try:
            print('Численный ответ = ', answer.p / answer.q)
        except AttributeError:
            print('Численный ответ = ', answer)
        try:
            print('Точность',  cur_acc.p / cur_acc.q)
        except AttributeError:
            print('Точность', cur_acc)
        break
        
    i +=1

Выражение


   ⎛3 ___⎞
cos⎝╲╱ x ⎠

0 порядок

Дифференциал


       ⎛3 ___⎞
y = cos⎝╲╱ x ⎠

При x = 0


...

Интегралыч


 7/3      5/3    
x      3⋅x       
──── - ────── + x
 56      10      

При a = 1


 7/3      5/3          
x      3⋅x          201
──── - ────── + x = ───
 56      10         280

Численный ответ =  0.7178571428571429
Точность 0.28214285714285714
